In [ ]:
from google.colab import drive
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import csv

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

In [ ]:
drive.mount('/drive') 

Mounted at /drive


In [ ]:
train_Les = np.load('/drive/My Drive/CAD_1_1/train_Les_hairRemoved.npz',allow_pickle=True)
train_NV  = np.load('/drive/My Drive/CAD_1_1/train_NV_hairRemoved.npz',allow_pickle=True)
val_Les   = np.load('/drive/My Drive/CAD_1_1/val_Les_hairRemoved.npz',allow_pickle=True)
val_NV    = np.load('/drive/My Drive/CAD_1_1/val_NV_hairRemoved.npz',allow_pickle=True)
test      = np.load('/drive/My Drive/CAD_1_1/test_hairRemoved.npz',allow_pickle=True)

In [ ]:
trainX_Les  = np.array(train_Les['arr_0'])
trainX_NV   = np.array(train_NV['arr_0'])
valX_Les    = np.array(val_Les['arr_0'])
valX_NV     = np.array(val_NV['arr_0'])
test       = np.array(test['test'])


In [ ]:
print(trainX_Les.shape)
print(trainX_NV.shape)
print(valX_Les.shape)
print(valX_NV.shape)
print(test.shape)

(2400, 450, 600, 3)
(2400, 450, 600, 3)
(600, 450, 600, 3)
(600, 450, 600, 3)
(1015, 450, 600, 3)


LBP SECOND TRIAL

In [ ]:
def LBP(image, channel=3, P=8, R=2, bins=8):
  lbp       = local_binary_pattern(image[:,:,0], P=P, R=R, method="uniform")
  lbp_0, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
  lbp       = local_binary_pattern(image[:,:,1], P=P, R=R, method="uniform")
  lbp_1, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
  lbp       = local_binary_pattern(image[:,:,2], P=P, R=R, method="uniform")
  lbp_2, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
  return lbp_0, lbp_1, lbp_2

In [ ]:
from skimage.feature import local_binary_pattern

lbp_Features_trainX_Les  = []
lbp_Features_trainX_NV   = []
lbp_Features_valX_Les    = []
lbp_Features_valX_NV     = []
lbp_Features_test        = []

for x in tqdm(trainX_Les):
  lbp_R, lbp_G, lbp_B    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2RGB),   channel=3)
  lbp_H, lbp_S, lbp_V    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2HSV),   channel=3)
  lbp_Y, lbp_Cr, lbp_Cb  = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB), channel=3)
  LBP_CGLF  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V,lbp_Y,lbp_Cr,lbp_Cb),axis=0)
  lbp_Features_trainX_Les.append(LBP_CGLF)




100%|██████████| 2400/2400 [17:12<00:00,  2.32it/s]


In [ ]:
for x in tqdm(trainX_NV):
  lbp_R, lbp_G, lbp_B    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2RGB),   channel=3)
  lbp_H, lbp_S, lbp_V    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2HSV),   channel=3)
  lbp_Y, lbp_Cr, lbp_Cb  = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB), channel=3)
  LBP_CGLF_trainX_Les  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V,lbp_Y,lbp_Cr,lbp_Cb),axis=0)
  lbp_Features_trainX_NV.append(LBP_CGLF_trainX_Les)
for x in tqdm(valX_Les):
  lbp_R, lbp_G, lbp_B    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2RGB),   channel=3)
  lbp_H, lbp_S, lbp_V    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2HSV),   channel=3)
  lbp_Y, lbp_Cr, lbp_Cb  = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB), channel=3)
  LBP_CGLF_trainX_Les  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V,lbp_Y,lbp_Cr,lbp_Cb),axis=0)
  lbp_Features_valX_Les.append(LBP_CGLF_trainX_Les)
for x in tqdm(valX_NV):
  lbp_R, lbp_G, lbp_B    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2RGB),   channel=3)
  lbp_H, lbp_S, lbp_V    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2HSV),   channel=3)
  lbp_Y, lbp_Cr, lbp_Cb  = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB), channel=3)
  LBP_CGLF_trainX_Les  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V,lbp_Y,lbp_Cr,lbp_Cb),axis=0)
  lbp_Features_valX_NV.append(LBP_CGLF_trainX_Les)
for x in tqdm(test):
  lbp_R, lbp_G, lbp_B    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2RGB),   channel=3)
  lbp_H, lbp_S, lbp_V    = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2HSV),   channel=3)
  lbp_Y, lbp_Cr, lbp_Cb  = LBP(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB), channel=3)
  LBP_CGLF_trainX_Les  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V,lbp_Y,lbp_Cr,lbp_Cb),axis=0)
  lbp_Features_test.append(LBP_CGLF_trainX_Les)

100%|██████████| 1015/1015 [07:25<00:00,  2.28it/s]


In [ ]:
lbp_Features_trainX_Les=np.array(lbp_Features_trainX_Les)
lbp_Features_trainX_NV=np.array(lbp_Features_trainX_NV)
lbp_Features_valX_Les=np.array(lbp_Features_valX_Les)
lbp_Features_valX_NV=np.array(lbp_Features_valX_NV)
lbp_Features_valX_NV=np.array(lbp_Features_valX_NV)
lbp_Features_test  = np.array(lbp_Features_test)
print(lbp_Features_trainX_Les.shape)
print(lbp_Features_trainX_NV.shape)
print(lbp_Features_valX_Les.shape)
print(lbp_Features_valX_NV.shape)
print(lbp_Features_test.shape)

(2400, 72)
(2400, 72)
(600, 72)
(600, 72)
(1015, 72)


In [ ]:
lbp_train=np.vstack((lbp_Features_trainX_Les,lbp_Features_trainX_NV))
lbp_val=np.vstack((lbp_Features_valX_Les,lbp_Features_valX_NV))
lbp_test=np.vstack(lbp_Features_test)

In [ ]:
import pandas as pd

trainData = pd.DataFrame(lbp_train)
trainData['labels']=0
trainData['labels'][0:2400]=1
valData = pd.DataFrame(lbp_val)
valData['labels']=0
valData['labels'][0:600]=1

testData = pd.DataFrame(lbp_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
trainData.drop(["labels"],axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
0,0.127538,0.058151,0.069378,0.078415,0.134916,0.075822,0.078018,0.177763,0.122788,0.058261,0.071004,0.077929,0.135425,0.076018,0.080255,0.178320,0.155407,0.045470,0.042095,0.042314,0.095612,0.086332,0.105991,0.226779,0.170021,0.038764,0.033597,0.032847,0.081514,0.085932,0.110924,0.246400,0.169884,0.040228,0.032119,0.031517,0.077597,0.088207,0.119683,0.240764,0.141769,0.051677,0.054119,0.058347,0.114883,0.082219,0.093182,0.203804,0.113508,0.059156,0.077727,0.089132,0.144329,0.073985,0.075627,0.166536,0.139870,0.028880,0.026693,0.024412,0.077111,0.099200,0.146815,0.257019,0.138015,0.032708,0.032184,0.029751,0.087831,0.098913,0.136403,0.244196
1,0.112729,0.061108,0.079449,0.088921,0.152086,0.070400,0.070963,0.164344,0.116121,0.059961,0.074821,0.079357,0.140000,0.073452,0.080050,0.176237,0.137932,0.050563,0.052539,0.051579,0.115141,0.083733,0.096178,0.212335,0.098181,0.031793,0.045834,0.046190,0.138341,0.100047,0.136231,0.203384,0.117375,0.046095,0.060818,0.065662,0.138519,0.084006,0.098056,0.189470,0.137748,0.050533,0.052744,0.051751,0.115218,0.083573,0.096089,0.212344,0.110741,0.061227,0.078270,0.084830,0.145881,0.072033,0.077043,0.169976,0.085624,0.025493,0.040477,0.039265,0.133363,0.107007,0.174207,0.194563,0.073695,0.025428,0.046720,0.044447,0.142596,0.109307,0.170039,0.187769
2,0.115313,0.060921,0.081259,0.088116,0.149159,0.071381,0.069760,0.164092,0.121114,0.058904,0.072293,0.074453,0.136830,0.076542,0.080240,0.179624,0.140862,0.050930,0.053570,0.052800,0.115360,0.083544,0.094228,0.208705,0.104812,0.030631,0.043093,0.040157,0.120041,0.095141,0.135241,0.230883,0.120041,0.044270,0.059458,0.067022,0.135446,0.083490,0.097739,0.192533,0.140868,0.050930,0.053570,0.052800,0.115357,0.083547,0.094231,0.208696,0.114868,0.061256,0.078753,0.082471,0.144836,0.073532,0.075493,0.168791,0.096433,0.030726,0.046527,0.044785,0.131413,0.094590,0.127973,0.227553,0.080290,0.026524,0.045481,0.044021,0.133757,0.105378,0.165721,0.198827
3,0.142951,0.054296,0.055899,0.057745,0.120287,0.079796,0.087988,0.201037,0.149058,0.052077,0.048436,0.047079,0.105393,0.082690,0.100314,0.214954,0.167093,0.037520,0.032121,0.030317,0.076539,0.089049,0.117550,0.249810,0.118510,0.036190,0.044255,0.043612,0.120453,0.094625,0.117674,0.224681,0.163588,0.046222,0.038717,0.037369,0.097016,0.085265,0.102670,0.229153,0.167084,0.037532,0.032119,0.030323,0.076536,0.089046,0.117553,0.249807,0.145070,0.052193,0.051879,0.052154,0.110913,0.082270,0.096447,0.209076,0.107609,0.030077,0.039197,0.038110,0.119167,0.104003,0.141778,0.220059,0.129893,0.034332,0.036326,0.033031,0.099858,0.093870,0.127804,0.244886
4,0.146400,0.055932,0.055742,0.056916,0.115431,0.080290,0.087603,0.201686,0.142723,0.054409,0.055298,0.057073,0.115052,0.082089,0.090403,0.202954,0.166397,0.040803,0.034676,0.032453,0.083490,0.086960,0.112284,0.242936,0.164693,0.040877,0.033028,0.030302,0.084397,0.089973,0.117807,0.238921,0.170376,0.045570,0.038006,0.037067,0.093209,0.083736,0.103153,0.228883,0.163754,0.042359,0.036904,0.034806,0.086827,0.086910,0.109775,0.238667,0.130557,0.056679,0.064400,0.067212,0.126634,0.079644,0.084560,0.190314,0.152187,0.035019,0.030240,0.027081,0.082942,0.093597,0.131861,0.247073,0.149022,0.036812,0.033778,0.030705,0.091076,0.091126,0.126379,0.241102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.155360,0.038714,0.039481,0.041627,0.093784,0.088216,0.107884,0.234933,0.145742,0.040921,0.044427,0.046199,0.102086,0.087769,0.105748,0.227108,0.161843,0.025579,0.025713,0.027452,0.067665,0.091902,0.137751,0.262095,0.135751,0.020184,0.020981,0.0

In [ ]:
trainDataShuffle=trainData.sample(frac=1)
valDataShuffle=valData.sample(frac=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rndst=RandomForestClassifier(n_estimators=100)
rndst.fit(trainData.drop(["labels"],axis=1), trainData['labels'])
predict=rndst.predict(valData.drop(["labels"],axis=1))
#rndst.score(predict,valData['labels'])
print(metrics.accuracy_score(valData['labels'],predict))



0.8225


In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC()
svc.fit(trainData.drop(["labels"],axis=1), trainData['labels'])
predict=svc.predict(valData.drop(["labels"],axis=1))
#rndst.score(predict,valData['labels'])
print(metrics.accuracy_score(valData['labels'],predict))

0.7808333333333334


In [ ]:
trainData.to_csv('/drive/My Drive/CAD_1_1/goodLBP_Train.csv',index=False)
valData.to_csv('/drive/My Drive/CAD_1_1/goodLBP_val.csv',index=False)

In [ ]:
testData

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
0,0.153804,0.045176,0.045043,0.044468,0.098750,0.087944,0.099336,0.225479,0.140856,0.049084,0.054039,0.054237,0.113393,0.084643,0.094519,0.209230,0.152530,0.044495,0.044471,0.043997,0.100012,0.087659,0.099464,0.227372,0.122649,0.018892,0.021052,0.020033,0.068492,0.106379,0.171259,0.271244,0.154246,0.038898,0.038053,0.039298,0.090169,0.092800,0.106560,0.239976,0.152530,0.044495,0.044471,0.043997,0.100012,0.087659,0.099464,0.227372,0.138287,0.050184,0.055366,0.055636,0.113567,0.085393,0.094584,0.206984,0.129893,0.020290,0.021799,0.021396,0.068684,0.103674,0.170877,0.263387,0.120584,0.022119,0.027010,0.027902,0.083283,0.104815,0.151932,0.262356
1,0.118124,0.051606,0.066234,0.078474,0.146447,0.077360,0.080690,0.181064,0.118030,0.048012,0.060839,0.071073,0.144201,0.080859,0.086604,0.190382,0.142293,0.038764,0.040560,0.042895,0.104640,0.089250,0.110018,0.231579,0.100652,0.027751,0.039473,0.040015,0.113606,0.110373,0.146693,0.221437,0.122308,0.039339,0.045034,0.044619,0.110344,0.095778,0.117191,0.225387,0.141440,0.038963,0.041212,0.043597,0.105360,0.089049,0.109745,0.230634,0.112133,0.048376,0.064895,0.076969,0.150658,0.079067,0.084356,0.183547,0.087182,0.023090,0.035446,0.034184,0.109799,0.115597,0.181010,0.213692,0.084711,0.022708,0.034471,0.031446,0.108936,0.112225,0.183081,0.222421
2,0.123689,0.065197,0.079182,0.077295,0.134308,0.074154,0.074711,0.171464,0.130951,0.058767,0.068530,0.065535,0.120916,0.079307,0.085630,0.190364,0.165973,0.041277,0.035772,0.031864,0.081197,0.088507,0.111535,0.243876,0.103274,0.033905,0.045529,0.042169,0.120930,0.098279,0.138444,0.217470,0.122068,0.055576,0.068474,0.078397,0.157099,0.071713,0.075387,0.171286,0.165973,0.041277,0.035772,0.031864,0.081197,0.088507,0.111535,0.243876,0.126107,0.062566,0.074018,0.069816,0.125070,0.078933,0.082249,0.181241,0.106987,0.039899,0.054083,0.052836,0.130228,0.092044,0.115123,0.208800,0.099855,0.037716,0.056661,0.059674,0.148439,0.088456,0.111416,0.197784
3,0.145239,0.047159,0.050009,0.050622,0.108234,0.084803,0.095564,0.218370,0.120720,0.054196,0.069641,0.077956,0.142308,0.075535,0.077932,0.181713,0.157099,0.038794,0.036575,0.035333,0.088619,0.091004,0.109579,0.242996,0.113973,0.021947,0.027295,0.026136,0.087268,0.105917,0.157656,0.259807,0.129597,0.053345,0.061754,0.067087,0.128726,0.082012,0.082113,0.195366,0.157099,0.038794,0.036575,0.035333,0.088619,0.091004,0.109579,0.242996,0.123434,0.052231,0.065108,0.073431,0.134213,0.079307,0.083893,0.188382,0.118184,0.025926,0.031724,0.030317,0.090551,0.104367,0.146827,0.252104,0.099096,0.025197,0.036044,0.034483,0.105319,0.109473,0.152833,0.237556
4,0.102892,0.066187,0.089914,0.104394,0.167070,0.063567,0.061923,0.144053,0.102578,0.063650,0.085615,0.096196,0.157790,0.068145,0.070504,0.155523,0.140664,0.047538,0.050293,0.053932,0.114261,0.083455,0.096664,0.213194,0.115573,0.048119,0.059656,0.063372,0.138053,0.083348,0.098634,0.193244,0.112720,0.061630,0.078166,0.089840,0.159378,0.069487,0.071413,0.157366,0.139665,0.048047,0.051585,0.055345,0.114521,0.083079,0.096187,0.211570,0.097375,0.063970,0.090439,0.104391,0.163381,0.065277,0.067010,0.148157,0.097739,0.041321,0.058747,0.062714,0.151668,0.090806,0.105372,0.191633,0.093591,0.038951,0.060954,0.065514,0.152839,0.091662,0.111150,0.185339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,0.157775,0.039268,0.039641,0.042329,0.095292,0.087624,0.102021,0.236050,0.152862,0.039985,0.041114,0.046059,0.099090,0.086074,0.103686,0.231129,0.171739,0.027073,0.024723,0.023724,0.068056,0.093384,0.124599,0.266702,0.111772,0.018643,0.023481,0.0

In [ ]:
testData.to_csv('/drive/My Drive/CAD_1_1/goodLBP_test.csv',index=False)